In [ ]:
# ArcGIS imports
from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.raster.analytics import copy_raster
import arcgis.env as arcgisenvs

# Pandas and numpy
import pandas as pd
import numpy as np

# Plotting tools
import matplotlib.pyplot as plt

# Raster tools
from osgeo import gdal

# datetime for metadata
from datetime import datetime

# Inputs

In [ ]:
EPSG_3857_wkt = 'PROJCRS["WGS 84 / Pseudo-Mercator", BASEGEOGCRS["WGS 84", ENSEMBLE["World Geodetic System 1984 ensemble", MEMBER["World Geodetic System 1984 (Transit)"], MEMBER["World Geodetic System 1984 (G730)"], MEMBER["World Geodetic System 1984 (G873)"], MEMBER["World Geodetic System 1984 (G1150)"], MEMBER["World Geodetic System 1984 (G1674)"], MEMBER["World Geodetic System 1984 (G1762)"], MEMBER["World Geodetic System 1984 (G2139)"], ELLIPSOID["WGS 84",6378137,298.257223563, LENGTHUNIT["metre",1]], ENSEMBLEACCURACY[2.0]], PRIMEM["Greenwich",0, ANGLEUNIT["degree",0.0174532925199433]], ID["EPSG",4326]], CONVERSION["Popular Visualisation Pseudo-Mercator", METHOD["Popular Visualisation Pseudo Mercator", ID["EPSG",1024]], PARAMETER["Latitude of natural origin",0, ANGLEUNIT["degree",0.0174532925199433], ID["EPSG",8801]], PARAMETER["Longitude of natural origin",0, ANGLEUNIT["degree",0.0174532925199433], ID["EPSG",8802]], PARAMETER["False easting",0, LENGTHUNIT["metre",1], ID["EPSG",8806]], PARAMETER["False northing",0, LENGTHUNIT["metre",1], ID["EPSG",8807]]], CS[Cartesian,2], AXIS["easting (X)",east, ORDER[1], LENGTHUNIT["metre",1]], AXIS["northing (Y)",north, ORDER[2], LENGTHUNIT["metre",1]], USAGE[ SCOPE["Web mapping and visualisation."], AREA["World between 85.06°S and 85.06°N."], BBOX[-85.06,-180,85.06,180]], ID["EPSG",3857]]'
raster_location = {
    "top_left_x": -13626617,
    "top_left_y": 4549726,
    "crs_wkid": 3857,
    "crs_wkt": EPSG_3857_wkt
}
pixel_size = 10.307360350155632
(pix_y, pix_x) = (100, 100)
random_raster_std_dev = 1.8
output_agol_folder_name = "Raster_Testing"
random_raster_file = "random_raster.tif"
arcgisenvs.verbose = True

# Generate Random Raster

In [ ]:
random_raster_df = pd.DataFrame(
    random_raster_std_dev*np.random.randn(pix_x*pix_y),
    columns=["pixvals"],
    dtype="float32"
)

# Visualize raster

In [ ]:
plt.imshow(random_raster_df.pixvals.to_numpy().reshape(pix_y, pix_x))
plt.colorbar()

# Save raster file

In [ ]:
# Create an empty GeoTIFF dataset
driver = gdal.GetDriverByName("GTiff")
dst_ds = driver.Create(
    random_raster_file,
    xsize=pix_x,
    ysize=pix_y,
    bands=1,
    eType=gdal.GDT_Float32,
)

# Set geotransform and spatial reference of the dataset
dst_ds.SetGeoTransform(
    [raster_location["top_left_x"], pixel_size, 0, raster_location["top_left_y"], 0, -pixel_size]
)
dst_ds.SetProjection(raster_location["crs_wkt"])

# Write raster pixels
dst_ds.GetRasterBand(1).WriteArray(
    random_raster_df.pixvals.to_numpy().reshape(pix_y, pix_x)
)

# Add metadata
dst_ds.SetMetadata(
    {
        "TIFFTAG_DOCUMENTNAME": "Random Raster",
        "TIFFTAG_IMAGEDESCRIPTION": "Random Raster generated from a DataFrame using GDAL",
        "TIFFTAG_DATETIME": f"{datetime.now().isoformat()}",
    }
)

# Close the dataset
dst_ds = None

# Connect to AGOL

In [ ]:
gis = GIS(profile="agol_profile")

# Create AGOL folder for analysis results

In [ ]:
output_agol_folder = gis.content.create_folder(output_agol_folder_name)

# Publish raster to AGOL

In [ ]:
random_raster_item = copy_raster(
        input_raster=random_raster_file,
        output_name="Random Raster",
        raster_type_name="Raster Dataset",
        context={"upload_properties": {"displayProgress": True}},
        folder=output_agol_folder,
        tiles_only=True,
    )

# Publish maps with raster added as a layer

## Method 1
* Create a local WebMap
* Add raster item from AGOL to local WebMap
* Save the WebMap to AGOL

In [ ]:
wm1 = WebMap()
wm1.basemap = "gray-vector"
wm1.add_layer(random_raster_item, options={"title":"Random Raster"})
wm1

In [ ]:
wm1_item = wm1.save(
    {
    "title": "Raster Test WebMap Method 1",
    "snippet": "Adding a raster item to a WebMap from a Jupyter Notebook",
    "description": "Started with a local WebMap (WebMap()), added the published raster item as a layer to the local WebMap (map.add_layer()), then used the WebMap.save() method to publish the local WebMap to AGOL",
    "tags": ["Automation"]
    },
    folder=output_agol_folder_name
)

wm1_item_wm = WebMap(wm1_item)
wm1_item_wm

## Method 2
  * Create a local map widget
  * Add raster item from AGOL to map widget
  * Save the map widget as a WebMap to AGOL

In [ ]:
map1 = gis.map()
map1.basemap = "gray-vector"
map1.add_layer(random_raster_item)
map1.zoom_to_layer(random_raster_item)
map1

In [ ]:
map1_wm_item = map1.save(
    {
    "title": "Raster Test WebMap Method 2",
    "snippet": "Adding a raster item to a WebMap from a Jupyter Notebook",
    "description": "Started with a local map widget (gis.map()), added the published raster item as a layer to the local map widget (map.add_layer()), then used the map.save() method to publish the local map widget as a WebMap",
    "tags": ["Automation"]
    },
    folder=output_agol_folder_name
)

map1_wm = WebMap(map1_wm_item)
map1_wm